In [3]:
#!/usr/bin/env python
# coding: utf-8

# # `foo.ipynb`

# In[1]:
def bar():
    return "bar"

# In[2]:
def has_ip_syntax():
    listing = get_ipython().getoutput("ls")
    return listing

# In[3]:
def whatsmyname():
    return __name__

Search Results:
             Name  Age
0      John Smith   34
1        Jane Doe   28
2    Mary Johnson   45
3  James Williams   30
4  Patricia Brown   38
